### Needed libraries

In [ ]:
!pip install jovian --upgrade --quiet
!pip install torch --upgrade --quiet
!pip install torchvision --upgrade --quiet

In [ ]:
import os    
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [ ]:
#Import relevant libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import jovian
import torchvision
import string
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import random_split
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset

## Hyperparameters and other constants

In [ ]:
# Hyperparameters
batch_size = 256
learning_rate = 1e-5

# Other constants
input_size = 28*28
num_classes = 26

## Load data

In [ ]:
dataset = pd.read_csv('archive/sign_mnist_train.csv')
testdataset = pd.read_csv('archive/sign_mnist_test.csv').values
print(dataset)
num_rows = dataset.shape[0]
# To map each label number to its corresponding letter
letters = dict(enumerate(string.ascii_uppercase))

In [ ]:
labels_array = dataset[:,0]
values_array = dataset[:,1:]
test_labels_array = testdataset[:,0]
test_values_array = testdataset[:,1:]


### Some examples from the train dataset

In [ ]:
fig, axis = plt.subplots(4,6,figsize=(10,10))
k = 0
for i in range(4):
    for j in range(6):
        pic1 = np.reshape(values_array[k], (28, 28))
        axis[i,j].imshow(pic1)
        axis[i,j].set_title("Letter: " + str(letters[labels_array[k].item()]))
        axis[i,j].axis('off')
        k+=1

## Adapt data to Torch library

In [ ]:
values = torch.from_numpy(values_array).float()
labels = torch.from_numpy(labels_array).long()
test_values = torch.from_numpy(test_values_array).float()
test_labels = torch.from_numpy(test_labels_array).long()

## Separate train, test and validation

In [ ]:
# Training validation & test dataset
dataset = TensorDataset(values, labels)
testdataset = TensorDataset(test_values, test_labels)

# Let's use 15% of our training dataset to validate our model
val_percent = 0.15
val_size = int(num_rows * val_percent)
train_size = num_rows - val_size
train_ds, val_ds = random_split(dataset, [train_size, val_size])

# Dataloaders
train_loader = DataLoader(train_ds, batch_size, shuffle=True)
val_loader = DataLoader(val_ds, batch_size*2)
test_loader = DataLoader(testdataset, batch_size*2)

### Confirm shuffle

In [ ]:
img, label = train_ds[0]
plt.imshow(img.reshape((28,28)), cmap = 'gray')
print("Letter: ", letters[label.item()])
# Confirm shuffle

## Logistic Regression Model Class

In [ ]:
class MnistModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(input_size, num_classes)
        
    def forward(self, xb):
        xb = xb.reshape(-1, 784)
        out = self.linear(xb)
        return out
    
    def training_step(self, batch):
        images, labels = batch 
        out = self(images)                  # Generate predictions
        loss = F.cross_entropy(out, labels) # Calculate loss
        return loss
    
    def validation_step(self, batch):
        images, labels = batch 
        out = self(images)                    # Generate predictions
        loss = F.cross_entropy(out, labels)   # Calculate loss
        acc = accuracy(out, labels)           # Calculate accuracy
        return {'val_loss': loss.detach(), 'val_acc': acc.detach()}
        
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        batch_accs = [x['val_acc'] for x in outputs]
        epoch_acc = torch.stack(batch_accs).mean()      # Combine accuracies
        return {'val_loss': epoch_loss.item(), 'val_acc': epoch_acc.item()}
    
    def epoch_end(self, epoch, result):
        print("Epoch [{}], val_loss: {:.4f}, val_acc: {:.4f}".format(epoch, result['val_loss'], result['val_acc']))
    
model = MnistModel()

### Needed functions to evaluate the model

In [ ]:
def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))

def evaluate(model, val_loader):
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)

def fit(epochs, lr, model, train_loader, val_loader, opt_func=torch.optim.SGD):
    history = []
    optimizer = opt_func(model.parameters(), lr)
    for epoch in range(epochs):
        # Training Phase 
        for batch in train_loader:
            loss = model.training_step(batch)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        # Validation phase
        result = evaluate(model, val_loader)
        #model.epoch_end(epoch, result)
        history.append(result)
    return history

### Train model with 8 different learning rates for 200 epochs (takes long time!)

In [ ]:
fig, axs = plt.subplots(4, 2,figsize=(9,9))
axis = axs.flat
learning_rates = [1,0.1,0.01,0.001,0.0001,1e-5,1e-06,1e-07]
epochs = 200
axis_index = 0
models = []
for lr in learning_rates:
    model = MnistModel()
    history = fit(epochs, lr, model, train_loader, val_loader)
    val_accuracies = [r['val_acc'] for r in history]
    axis[axis_index].plot(val_accuracies, '-x')
    axis[axis_index].set(xlabel='epoch', ylabel='accuracy')
    axis[axis_index].set_title('Accuracy vs. No. of epochs with lr=' + str(lr))
    axis_index+=1
    models.append(model)
fig.tight_layout()
plt.show()

### Evaluate model 5 (lr=1e-05) and plot accuracies

In [ ]:
model_2 = MnistModel()
evaluate(model_2, val_loader)
history_2 = fit(200, 1e-05, model_2, train_loader, val_loader)

accuracies_2 = [r['val_acc'] for r in history_2]
plt.plot(accuracies_2, '-x')
plt.xlabel('epoch')
plt.ylabel('accuracy')
plt.title('Accuracy vs. No. of epochs');


### Evaluate with test dataset

In [ ]:
evaluate(model_2, test_loader)
def evaluate_stats(model, test_loader, val_loader):
    result_validation = evaluate(model, val_loader)
    result_test = evaluate(model, test_loader)
    print("\t\tLoss\tAccuracy")
    print("VS\t",round(result_validation["val_loss"],4),round(result_validation["val_acc"],4))
    print("Train Set\t",round(result_test["val_loss"],4),round(result_test["val_acc"],4))
evaluate_stats(model_2, test_loader, val_loader)

### Function to predict image

In [ ]:
def predict_image(img, model):
    xb = img.unsqueeze(0)
    yb = model(xb)
    _, preds  = torch.max(yb, dim=1)
    return preds[0].item()

In [ ]:
import random

## Plot predictions for custom images

In [ ]:
from PIL import Image
import matplotlib.image as img
import pandas as pd

fig, axes = plt.subplots(3, 2, figsize = (8, 6))
axes = axes.ravel()
labels = [19, 0, 21, 14, 11, 23]
for i in np.arange(0, 6):
    image = np.array(Image.open('image_'+str((i+1))+'.jpg').convert('L').resize((28,28)))
    image = image.reshape(28*28,1)
    image = image.T / 255
    ntest_labels_array = np.asarray([labels[i]]).T
    ntest_values_array = image
    ntest_values = torch.from_numpy(ntest_values_array).float()
    ntest_labels = torch.from_numpy(ntest_labels_array).long()
    newtestdataset = TensorDataset(ntest_values, ntest_labels)
    img, label = random.choice(newtestdataset)
    axes[i].imshow(img.reshape(28,28))
    title = 'Label:' + str(letters[label.item()]) + ', Predicted:' + str(letters[predict_image(img, models[0])])
    axes[i].set_title(title)
    axes[i].axis('off')
plt.subplots_adjust(wspace=0.5)